## For assemblies requiring extension PCR

Since this is the least used module for the assembly, it seems a bit scatterred. 

In [4]:
import os
import pandas as pd
import numpy as np
import re
import sys
from string import ascii_uppercase
import glob
from natsort import natsorted
import math

os.getcwd()




'/Users/nilmani/Desktop/Python/Plant_Picklist'

**This is modified module to run PCR set up for assemblies that require Extension PCR.**

STEP 1: 

-  create consolidated PCR files for all PCRs required for assembly

In [5]:
def write_to_PCR_folder(filename, dataframe_object):
    
    curr_dir = os.getcwd()
    subdirectory = 'PCR_files'
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        os.chdir('..')


def Read_Original_Files ():
    
    column_names = ["Source Well", "Transfer Volume", "Destination Well", "Primer Name", "Ta"]
    df_temp = pd.DataFrame(columns = column_names)
    df_pick = pd.DataFrame(columns = column_names)

    file_list = glob.glob('**/*Pos_Primers_Guides_*.csv')
    file_list = natsorted(file_list)
    
    curr_dir = os.getcwd()
        
    for filename in file_list:
        
        file_path = os.path.join(curr_dir, filename)
        df_primers = pd.read_csv(file_path)
        
        df_temp = Create_PCR_Picklist(df_primers)
        df_pick = df_pick.append(df_temp)

                
    filename = 'All_PCRs_ECHO_setup.csv'
    write_to_PCR_folder(filename, df_pick)
       

def Create_PCR_Picklist(df_primers):
    
    column_names = ["Source Well", "Transfer Volume", "Destination Well", "Primer Name", "Ta", "PCR size"]
    column_names.append("Template_Name")
    column_names.append("Extension_Template")
    df_picklist = pd.DataFrame(columns = column_names)
    
    length = len(df_primers)

    list2= [] # For Fwd primer
    list3 = [] # For reverse primer

    # use this to break the loop, as for the last index, there won't anything after it
    file_length = (len(df_primers) - 1)
        
    counter = int(length/2)
    Vol = 1000
    
    for index, row in df_primers.iterrows():
        
        if index == file_length:
            break

        # The rev primer should come after Fwd primer
        Fwd_primer = df_primers['Name'].loc[index]
        Rev_primer = df_primers['Name'].loc[index+1]
        
        # This requires that the file has fwd primer and then the rev primer
        isFwd = ((re.search("[F][P]", Fwd_primer))) 
        isRev = ((re.search("[R][P]", Rev_primer))) 

        if not isFwd:
            continue

        # To check if both primers belong to same PCR fragment 
        Fragment_fwd = re.findall('[F][P]_(\d{1,2})', Fwd_primer)
        Fragment_rev = re.findall('[R][P]_(\d{1,2})', Rev_primer)
        
        if isFwd and isRev and (Fragment_fwd == Fragment_rev):

            s_well_1 = df_primers['384well_pos_ECHO'].loc[index]
            s_well_2 = df_primers['384well_pos_ECHO'].loc[index+1]
            
            Ta_F = df_primers['Ta'].loc[index]
            Ta_R = ''
            Temp_len_f = df_primers['Template Length'].loc[index]
            Temp_len_r = ''
            Temp_name_F = df_primers['Template_Name'].loc[index]
            Temp_name_R = ''
            Temp_ext_F = df_primers['Extension_Template'].loc[index]
            Temp_ext_R = ''

            d_well = '' # will add the destination well using another function

            list2 = [s_well_1, Vol, d_well, Fwd_primer, Ta_F, Temp_len_f, Temp_name_F, Temp_ext_F]
            list3 = [s_well_2, Vol, d_well, Rev_primer, Ta_R, Temp_len_r, Temp_name_R, Temp_ext_R]

            a_series = pd.Series(list2, index = df_picklist.columns)
            b_series = pd.Series(list3, index = df_picklist.columns)
            df_picklist = df_picklist.append(a_series, ignore_index=True)
            df_picklist = df_picklist.append(b_series, ignore_index=True)

        else:
            continue
        
    return df_picklist


if __name__ == '__main__':
    Read_Original_Files()


File written:-    All_PCRs_ECHO_setup.csv


**This one takes the All PCR files, divides it into Extension PCRs and Main PCRs. Then creates a files for position of all templates.**

-  Divide the PCRs by whether these are Extension PCR or main PCR for PfAgo assembly

In [7]:
def write_to_PCR_folder(filename, dataframe_object):
    
    curr_dir = os.getcwd()
    subdirectory = 'PCR_files'
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        os.chdir('..')



def Separate_extension_PCR(filename):
    
    All_PCR_df = pd.read_csv(filename)
    
    column_list = []
    for column in All_PCR_df:
        column_list.append(column)
        
    Extension_df = pd.DataFrame(columns = column_list)
    Main_PCR_df = pd.DataFrame(columns = column_list)
    
    
    for index, row in All_PCR_df.iterrows():
        
        if index == len(All_PCR_df) - 1:
            break
        
        Fwd_primer = All_PCR_df['Primer Name'].loc[index]
        Rev_primer = All_PCR_df['Primer Name'].loc[index+1]
        
        isEFwd = ((re.search("[E][F][P]", Fwd_primer))) 
        isERev = ((re.search("[E][R][P]", Rev_primer))) 
        
        isFwd = ((re.search("[F][P]", Fwd_primer))) 
        isRev = ((re.search("[R][P]", Rev_primer))) 

        if not isEFwd and not isERev and isFwd and isRev:
            
            # To check if both primers belong to same PCR fragment 
            Fragment_fwd = re.findall('[F][P]_(\d{1,2})', Fwd_primer)
            Fragment_rev = re.findall('[R][P]_(\d{1,2})', Rev_primer)
            
            if Fragment_fwd == Fragment_rev:
                Main_PCR_df = Main_PCR_df.append(All_PCR_df.loc[index], ignore_index = True)
                Main_PCR_df = Main_PCR_df.append(All_PCR_df.loc[index+1], ignore_index = True)
                continue
                
        if isEFwd and isERev:
                        
            # To check if both primers belong to same PCR fragment 
            Fragment_fwd = re.findall('[E][F][P]_(\d{1,2})', Fwd_primer)
            Fragment_rev = re.findall('[E][R][P]_(\d{1,2})', Rev_primer)
            
            if Fragment_fwd == Fragment_rev:
                Extension_df = Extension_df.append(All_PCR_df.loc[index], ignore_index = True)
                Extension_df = Extension_df.append(All_PCR_df.loc[index+1], ignore_index = True)
                continue
            

    filename = 'Main_PCRs_Plant.csv'
    write_to_PCR_folder(filename, Main_PCR_df)
    filename = 'Extension_PCRs_Plant.csv'
    write_to_PCR_folder(filename, Extension_df)
    print('Finished')

    
def Template_positions():
    # Add the templates in last two rows O1 - P24
    Well_list = []
    for item in ['N','O', 'P']:
        for i in range(24):
            well = str(item)+str(i+1)
            Well_list.append(well)
            
    return Well_list
    
    
def All_unique_templates(filename):
    
    All_PCR_df = pd.read_csv(filename)
    df_unique = pd.DataFrame(columns = ['Template_Name', 'Well'])
    All_PCR_df = All_PCR_df.drop_duplicates(subset=['Template_Name'])
    
    Well_list = Template_positions()
    
    list_original = []
    list_Extension  = []
    counter = 0
    Template_wells = []
    counter  = 0
    
    print(len(All_PCR_df))
    
    for index, row in All_PCR_df.iterrows():
        
        Name = str(All_PCR_df['Template_Name'].loc[index])
    
        if Name == 'nan':
            continue
        
        isOriginal = ((re.search('\d{1,2}[J][S]', Name)))
        
        if not isOriginal:
            list_original.append(Name)
            
        elif isOriginal:
            list_Extension.append(Name)

            
        Template_wells.append(Well_list[counter])
        counter = counter + 1
        
        #if Name == 'Backbone':
         #   for i in range(2):
          #      list_original.append(Name)
           #     Template_wells.append(Well_list[counter])   
            #    counter = counter + 1
            
    
    for item in list_Extension:
        list_original.append(item)
    
    df_unique['Template_Name'] = list_original
    df_unique['Well'] = Template_wells
    
    filename = 'All_Templates_Plants.csv'
    write_to_PCR_folder(filename, df_unique,)
    
    
    
def Read_allPCR_Files():
    file_list = glob.glob('**/*All_PCRs_ECHO_setup*.csv')
    file_list = natsorted(file_list)
    
    for filename in file_list:
        print(filename)
        Separate_extension_PCR(filename)
        All_unique_templates(filename)
        


if __name__ == '__main__':
    Read_allPCR_Files()

PCR_files/All_PCRs_ECHO_setup.csv
File written:-    Main_PCRs_Plant.csv
File written:-    Extension_PCRs_Plant.csv
Finished
52
File written:-    All_Templates_Plants.csv


### Add wells of 96well to PCR plate 

In [8]:
def Find_next_same_PCRs(df_PCR_copy, start_ind):
    
    number_same_PCR = 1
    flag = True
    
    while(flag  == True):
        Curr_primer = df_PCR_copy["Primer Name"].loc[start_ind]
        Next_primer = df_PCR_copy["Primer Name"].loc[start_ind+1]
        Curr_name = re.findall('^\d{1,2}[A-Za-z]{1,}', Curr_primer)
        Next_name = re.findall('^\d{1,2}[A-Za-z]{1,}', Next_primer)
        
        if Curr_name == Next_name:
            start_ind = start_ind + 1
            number_same_PCR = number_same_PCR + 1
            continue
        else:
            flag = False 
            
        
    return number_same_PCR


def Wellplate96_PCR():
    
    # we will keep wells A1, A2 and H11, H12 empty
    # For running on FA, to add ladder or some other standrd
    # In each 96well plate, we will only set 92 PCRs
    Wellplate96 = []
    Skip_wells = ['A1', 'A2', 'H11', 'H12']

    for letter in ascii_uppercase:
        for j in range(1,13):            
            dest = letter+str(j)
            if dest in Skip_wells:
                continue
            Wellplate96.append(dest)
        if letter == 'H':
            break
            
            
    return Wellplate96

    
def Add_96well_dest(filename):
    
    df_PCR = pd.read_csv(filename)

    Wellplate96 = Wellplate96_PCR()
            
    Plate_counter = 1   
    Well_index_96well = 0
    Number_of_wells = 92
    start_ind = 0
    initial_cond = 183

    
    df_PCR = df_PCR.reset_index(drop = True)
    column_list = []
    for column in df_PCR:
        column_list.append(column)
    df_temp = pd.DataFrame(columns = column_list)
    Dest_well_list = []
    
    even_counter = 0
    for index, row in df_PCR.iterrows():

        if start_ind == 0:
            number_same_PCR = Find_next_same_PCRs(df_PCR, start_ind)
            start_ind = start_ind + number_same_PCR
            Indices_required = int(index + number_same_PCR)

        elif start_ind == index:
            if len(df_PCR) < start_ind: 
                number_same_PCR = Find_next_same_PCRs(df_PCR, start_ind)
            start_ind = start_ind + number_same_PCR 
            Indices_required = int(index + number_same_PCR)
        
        #check if all of the next ones can come to same 96well plate 
        if Indices_required > initial_cond:
            initial_cond = initial_cond + 183

            df_temp.loc[:,'Destination Well'] = Dest_well_list
            filename = 'ECHO_PCR_setup_plate_'+str(Plate_counter)+'.csv'
            print('Number of PCRs in this plate ----> ', int(len(df_temp)/2))
            write_to_PCR_folder(filename, df_temp)
            
            Dest_well_list = []
            del df_temp
            df_temp = pd.DataFrame(columns = column_list)
            Plate_counter = Plate_counter + 1
            Well_index_96well = 0
        
        df_temp = df_temp.append(df_PCR.loc[index], ignore_index = True)
        even_counter = even_counter + 1
        
        if even_counter % 2 != 0:
            dest_well = Wellplate96[Well_index_96well]
            Well_index_96well = Well_index_96well + 1

            for i in range(2):
                Dest_well_list.append(dest_well)
                
        if len(df_PCR) == (index + 1):
            df_temp.loc[:,'Destination Well'] = Dest_well_list
            
            new_file = filename.split('/')
            new_file = new_file[1]

            filename_New = 'ECHO_PCR_setup_' + new_file
            print('Number of PCRs in this plate ----> ', int(len(df_temp)/2))
            write_to_PCR_folder(filename_New, df_temp)

    print('Number of 96well plates required for setting up PCR ----->  ', Plate_counter)

    
def Read_allPCR_Files():
    Extension_PCR = glob.glob('**/*Extension_PCRs_Plant*.csv')
    Main_PCR = glob.glob('**/*Main_PCRs_Plant*.csv')
    
    for filename in Extension_PCR:
        Add_96well_dest(filename)
        print(filename)
        
        
    for filename in Main_PCR:
        Add_96well_dest(filename)
        print(filename)
    

if __name__ == '__main__':
    Read_allPCR_Files()
    
    

Number of PCRs in this plate ---->  42
File written:-    ECHO_PCR_setup_Extension_PCRs_Plant.csv
Number of 96well plates required for setting up PCR ----->   1
PCR_files/Extension_PCRs_Plant.csv
Number of PCRs in this plate ---->  83
File written:-    ECHO_PCR_setup_Main_PCRs_Plant.csv
Number of 96well plates required for setting up PCR ----->   1
PCR_files/Main_PCRs_Plant.csv


## Master mix is in a separate plate 

`same as the main module`<br>
The primers and the templates will be on one single plate. <br>
The primers and templates can be re-used. <br>
The master mix will be in a different plate. <br>

**We need to transfer the master mix using Fluent.**

In [54]:
def fluent_picklist():
        
    column_names = ["Source plate", "S-Well position", "Destination plate", "D-Well position", "Volume", "Primer"]
    df_picklist = pd.DataFrame(columns = column_names)
    
    return df_picklist

    
def Fluent_96well_format():
    
    Wellplate96 = []
    counter = 0
    
    for j in range(1,13):
        for letter in ascii_uppercase:
            counter = counter + 1
            dest = str(counter) + ' ' +letter+str(j)
            Wellplate96.append(dest)
            
            if letter == 'H':
                break
            
    return Wellplate96


def Find_well_number(d_well, Wellplate96):
    
    for item in Wellplate96:
        well_n = item.split(' ')    

        if d_well == well_n[1]:
            well_number = well_n[0]    
            
    return well_number 
    

def read_PCR_files():
    
    file_list = glob.glob('**/*ECHO_PCR_setup_*')
    file_list = natsorted(file_list)
    
    counter = 0
    curr_dir = os.getcwd()

    for file in file_list:
        
        counter = counter + 1

        PCR_plate = os.path.join(curr_dir, file)
        df_96pcr_plate = pd.read_csv(PCR_plate) 
        
        master_mix_picklist_ECHO(df_96pcr_plate, counter)    
    
    
def Source_Wells(total_PCRs, counter):
    
    # Each well <1.5ml master mix 
    # Distribute 47ul master mix to maximum 36 wells from one well 
    total_PCRs = total_PCRs + 2
    Source_well_n = int(math.ceil((total_PCRs / 35)))  # Number of wells with master mix 
    
    df_source_well = pd.DataFrame()
    print('Number of wells with master mix :- ', Source_well_n)
    
    Master_mix_volume = []
    well_list = []
    S_well_list = []
    
    for i in range(Source_well_n):
        S_well_list.append(i+1)
        
        if (i+1) < Source_well_n:
            Master_mix_volume.append(1700)
            well_list.append(i+1)
            
        else:
            volume = 1700 * ((total_PCRs % 35)/35)
            volume = math.ceil(volume / 10)  * 10
            Master_mix_volume.append(volume)
            well_list.append(i+1)

    
    Total_reaction_vol = sum(Master_mix_volume)
    total_PCRs = math.ceil(Total_reaction_vol / 47)
                
    well_list.append('Total volume') 
    Master_mix_volume.append(Total_reaction_vol)
    
    Buffer_5x = math.ceil(total_PCRs * 10)
    dNTP_10uM = math.ceil(total_PCRs )
    Polymerase = math.ceil(total_PCRs )
    Water_to_50ul = math.ceil(total_PCRs * 35)
    
    well_list.append('\n PCR reaction setup') 
    Master_mix_volume.append('\n Volumes')
    
    well_list.append('5x_PCR_buffer') 
    Master_mix_volume.append(Buffer_5x)
    
    well_list.append('dNTP (10uM)') 
    Master_mix_volume.append(dNTP_10uM)

    well_list.append('DNA Polymerase') 
    Master_mix_volume.append(Polymerase)

    well_list.append('Water') 
    Master_mix_volume.append(Water_to_50ul)
    
    Sum_1 = Buffer_5x + dNTP_10uM + Polymerase + Water_to_50ul
                
    well_list.append('SUM') 
    Master_mix_volume.append(Sum_1)
    
    
    df_source_well['Well number'] = well_list
    df_source_well['Volume'] = Master_mix_volume
    
    filename = 'Master_mix_distrib_deepwell_forFluent_'+str(counter)+'.csv'
    write_to_PCR_folder(filename, df_source_well)
    
    return S_well_list    
    

def master_mix_picklist_ECHO(df_96pcr_plate,  counter):
    
    df_temp = fluent_picklist()
    df_temp2 = fluent_picklist()
    Wellplate96 = Fluent_96well_format()

    list1 = []
    volume = 47
    source_plate   = 'Master_mix[001]'
    
    total_PCRs = len(df_96pcr_plate)/2
    #print('total_PCRs:', total_PCRs)
    S_well = Source_Wells(total_PCRs, counter)
        
    Dest_plate = "PCR_reaction[001]"
    
    Skip_counter = 0
    
    for index, row in df_96pcr_plate.iterrows():
                 
        Skip_counter = Skip_counter + 1
        if Skip_counter % 2 == 0:
            continue
            
        S_well_index = math.floor(index / 70)
        S_well_number = S_well[S_well_index]
        
        d_well = df_96pcr_plate['Destination Well'].loc[index]
        Primer =  df_96pcr_plate['Primer Name'].loc[index]
        
        D_well_number = Find_well_number(d_well, Wellplate96)
        list1 = [source_plate, S_well_number, Dest_plate, D_well_number, volume, Primer]
         
            
        a_series = pd.Series(list1, index = df_temp.columns)
        df_temp = df_temp.append(a_series, ignore_index=True)
       
    filename = 'Picklist_PCR_master_mix_on_Fluent_'+str(counter)+'.csv'
    write_to_PCR_folder(filename, df_temp)

    
if __name__ == '__main__':
    
    read_PCR_files()

Number of wells with master mix :-  2
File written:-    Master_mix_distrib_deepwell_forFluent_1.csv
File written:-    Picklist_PCR_master_mix_on_Fluent_1.csv
Number of wells with master mix :-  3
File written:-    Master_mix_distrib_deepwell_forFluent_2.csv
File written:-    Picklist_PCR_master_mix_on_Fluent_2.csv


## Add Template in ECHO


Find template in original template location file
Re-write the `Final_Templates_Plants_384well.csv` files. 



In [55]:
def write_to_PCR_folder2(filename, dataframe_object):
    
    curr_dir = os.getcwd()
    subdirectory = 'PCR_files'
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        os.chdir('..')
        

def Read_template_PCR_files():

    File_template = glob.glob('**/*Final_Templates_Plants_384well*')
    print(File_template)
    df_template = pd.read_csv(File_template[0])
    print(df_template)

    PCR_files = glob.glob('**/*ECHO_PCR_setup_*')
    
    for filename in PCR_files:
        add_template(filename,  df_template)
    

def add_template(PCR_file,  df_template):
    
    df_PCR = pd.read_csv(PCR_file)
    df_temp = pd.DataFrame()
    
    column_list = []
    for column in df_PCR:
        column_list.append(column)
        
    df_temp = pd.DataFrame(columns = column_list)
        
    List2 = []
    Volume = 1000

    for index, row in df_PCR.iterrows():            
            
        df_temp = df_temp.append(df_PCR.loc[index], ignore_index = True)
        
        if index == 0:
            continue
        
        template = str(df_PCR['Template_Name'].loc[index-1])
        
        if template == 'nan':
            continue
        

        match = df_template[df_template['Template_Name'].str.match(template)]
        if match.empty:
            print('Template not found -->  ', template)
            continue
            
        Template_found = False
        for index_2, row_2 in match.iterrows():
            Template_database = match['Template_Name'].loc[index_2]
            
            if Template_database == template:
                Template_source_Well = match['384_Well'].loc[index_2]
                Template_found = True
                
        if Template_found == True:
            Dest = df_PCR['Destination Well'].loc[index-1]
            List2 = [ Template_source_Well, Volume, Dest, '', '', '','', '']
            
            a_series = pd.Series(List2, index = df_temp.columns)
            df_temp = df_temp.append(a_series, ignore_index=True)

            
    PCR_f = PCR_file.split('/')
    PCR_f = PCR_f[1]
    filename = 'Picklist_'+ PCR_f
    write_to_PCR_folder2(filename, df_temp)
        
    
if __name__ == '__main__':
    
    Read_template_PCR_files()

['PCR_files/Final_Templates_Plants_384well.csv']
         Template_Name 384_Well
0             Backbone       N1
1               pSAG12       N2
2                 LEC2       N3
3                pCaMV       N4
4                 tNOS       N5
5              MmDGAT2       N6
6              AtDGAt1       N7
7                 PAP2       N8
8                 NTT1       N9
9      HA_Backbone_1JS      N10
10         LEC2_HA_1JS      N11
11    myc_Backbone_2JS      N12
12         LEC2_HA_2JS      N13
13         HA_tNOS_2JS      N14
14     MmDGAT2_myc_2JS      N15
15    myc_Backbone_3JS      N16
16         LEC2_HA_3JS      N17
17         HA_tNOS_3JS      N18
18     AtDGAT1_myc_3JS      N19
19    myc_Backbone_4JS      N20
20         PAP2_HA_4JS      N21
21         HA_tNOS_4JS      N22
22     MmDGAT2_myc_4JS      N23
23    myc_Backbone_5JS      N24
24         PAP2_HA_5JS       O1
25         HA_tNOS_5JS       O2
26     MmDGAT2_myc_5JS       O3
27    myc_Backbone_6JS       O4
28         NTT1_HA_6JS 

In [10]:
import glob
import pandas as pd
import os

print(os.getcwd())

File_template = glob.glob('**/*Picklist_ECHO_PCR_Main_PCR_Plant_07192021*')
print(File_template)

for filename in File_template:
    df_template = pd.read_csv(filename)
    v = df_template['Source Well'].value_counts()
    print(filename, '\n', v.to_string())

/Users/nilmani/Desktop/Python/Plant_Picklist
['PCR_files/Picklist_ECHO_PCR_Main_PCR_Plant_07192021.csv']
PCR_files/Picklist_ECHO_PCR_Main_PCR_Plant_07192021.csv 
 N1     20
N4     15
N2      6
A2      5
A13     5
A3      5
A12     5
A1      4
B8      4
B9      4
D19     3
D18     3
A5      3
A7      3
A4      3
A16     3
B22     3
A6      3
B2      2
D22     2
A8      2
E8      2
C8      2
B4      2
C21     2
E2      2
B24     2
D13     2
B1      2
C11     2
B12     2
C6      2
D11     2
D24     2
C7      2
A15     2
B13     2
D21     2
C1      2
D23     2
D12     2
D20     2
A9      2
A14     2
E1      2
B23     2
C2      2
E5      1
C23     1
C17     1
D16     1
N16     1
C10     1
E14     1
D15     1
O19     1
D6      1
C14     1
N19     1
P3      1
N21     1
O13     1
E7      1
N24     1
C9      1
D1      1
O23     1
D2      1
O8      1
O7      1
E20     1
O9      1
N18     1
C19     1
O10     1
D17     1
B17     1
C24     1
E3      1
N23     1
A11     1
E21     1
P1      1
N22    

## Add extension PCR products back to 384-well plate template position

In [ ]:
#Restart Kernel Please
import os
os._exit(00)

In [1]:
import os
import pandas as pd
import numpy as np
import re
import sys
from string import ascii_uppercase
import glob
from natsort import natsorted
import math

os.getcwd()

'/Users/nilmani/Desktop/Python/Plant_Picklist'

In [9]:
def write_to_PCR_folder(filename, dataframe_object):
    
    curr_dir = os.getcwd()
    subdirectory = 'PCR_files'
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print("File written:-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        os.chdir('..')


def find_Template_position(Temp_name, Ext_PCR_df):
    
    Row_Well = Ext_PCR_df[Ext_PCR_df['Extension_Template'].str.match(Temp_name)]
    Row_Well = Row_Well.reset_index(drop=True)

    Wellname = ''
        
    for index, row in Row_Well.iterrows():
        Template_search = str(Row_Well['Extension_Template'].loc[index])
        
        if Temp_name == Template_search:
            Wellname  = str(Row_Well['Destination Well'].loc[index])

            
    return Wellname
        

def Transfer_template_to_384well(file1, file2):
    
    Template_df = pd.read_csv(file1)
    Ext_PCR_df = pd.read_csv(file2)
    
    Ext_PCR_df = Ext_PCR_df.dropna(subset = ['Extension_Template'])
    
    column_list = ['Source Well', 'Volume' ,'Destination Well', 'Template_name']
    Picklist_df = pd.DataFrame(columns = column_list)
    list2 = []
    Volume = 60
    
    for index, row in Template_df.iterrows():
        
        Temp_name = str(Template_df['Template_Name'].loc[index])
        Dest_well = str(Template_df['384_Well'].loc[index])
        Source_Well = find_Template_position(Temp_name, Ext_PCR_df)
                
        if Temp_name == 'nan' or Source_Well == '':
            continue
        
        list2 = [Source_Well, Volume, Dest_well, Temp_name]
        
        a_series = pd.Series(list2, index = Picklist_df.columns)
        Picklist_df = Picklist_df.append(a_series, ignore_index=True)
                
    filename = 'Picklist_ext_template_to_384ECHO.csv'
    write_to_PCR_folder(filename, Picklist_df)
    

def Read_Temp_Ext_PCR_files():
    Template = glob.glob('**/*Final_Templates_Plants_384well*')
    Ext_PCR = glob.glob('**/*Purified_Extension_PCRs_Plants_edited*')

    print(Template, Ext_PCR)
    for file1, file2 in zip(Template, Ext_PCR):
        
        print(file1, '\n', file2)
        Transfer_template_to_384well(file1, file2)
        
    
if __name__ == '__main__':
    
    Read_Temp_Ext_PCR_files()

['PCR_files/Final_Templates_Plants_384well.csv'] ['PCR_files/Purified_Extension_PCRs_Plants_edited.csv']
PCR_files/Final_Templates_Plants_384well.csv 
 PCR_files/Purified_Extension_PCRs_Plants_edited.csv
File written:-    Picklist_ext_template_to_384ECHO.csv


In [8]:

file_list = glob.glob("**/*Purified_Extension_PCRs_Plants*")

for filename in file_list:
    DF = pd.read_csv(filename)
    
DF = DF.dropna(subset=['Extension_Template'])
    
DF.to_csv("Purified_Extension_PCRs_Plants_edited.csv", index = False)

